In [ ]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

tf.config.get_visible_devices()

In [ ]:
# Import the dataset.
training_df = pd.read_csv(filepath_or_buffer="https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")

# Scale the label.
training_df["median_house_value"] /= 1000.0

# Print the first rows of the pandas DataFrame.
training_df.head()

In [ ]:
# Get statistics on the dataset.
training_df.describe()

In [ ]:
#testing correlations
corr = training_df.corr()
corr

In [ ]:
def plot_df_corr(df):
    correlation = df.corr()
    # plot the correlation matrix as a heatmap
    fig, ax = plt.subplots()
    im = ax.matshow(correlation)

    # add x and y axis labels
    ax.set_xticks(np.arange(len(df.columns)))
    ax.set_yticks(np.arange(len(df.columns)))

    ax.set_xticklabels(df.columns)
    ax.set_yticklabels(df.columns)
    plt.xticks(rotation=90)


    # add a colorbar legend
    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.set_ylabel("Correlation", rotation=-90, va="bottom")

    plt.show()